In [7]:
import os
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential

keyVaultName = "gios-keyvault"
KVUri = f"https://gios-keyvault.vault.azure.net/"

credential = DefaultAzureCredential()
client = SecretClient(vault_url=KVUri, credential=credential)

print(f"Retrieving your secrets from KV_NAME.")

os.environ["AZURE_OPENAI_API_KEY"] = client.get_secret("AZURE-OPENAI-API-KEY").value
os.environ["AZURE_OPENAI_ENDPOINT"] = client.get_secret("AZURE-OPENAI-ENDPOINT").value

Retrieving your secrets from KV_NAME.


In [8]:
import openai
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
from transformers import GPT2TokenizerFast

API_KEY = os.getenv("AZURE_OPENAI_API_KEY") 
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT") 

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2022-12-01"

url = openai.api_base + "/openai/deployments?api-version=2022-12-01"

r = requests.get(url, headers={"api-key": API_KEY})

print(r.text)

{
  "data": [
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-davinci-002",
      "owner": "organization-owner",
      "id": "Davinci-002",
      "status": "succeeded",
      "created_at": 1677170437,
      "updated_at": 1677170437,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-ada-001",
      "owner": "organization-owner",
      "id": "ada-001",
      "status": "succeeded",
      "created_at": 1677170559,
      "updated_at": 1677170559,
      "object": "deployment"
    }
  ],
  "object": "list"
}


In [12]:
df = pd.read_csv("data/bill_sum_data.csv") # example: df = pd.read_csv("c:\\test\\bill_sum_data.csv")df
df

,Unnamed: 0,bill_id,text,summary,title,text_len,sum_len
0,0,110_hr37,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...,8494,321
1,1,112_hr2873,SECTION 1. SHORT TITLE.\n\n This Act may be...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,6522,1424
2,2,109_s2408,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...,6154,463
3,3,108_s1899,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...,19853,1400
4,4,107_s1531,SECTION 1. SHORT TITLE.\n\n This Act may be...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...,6273,278
5,5,107_hr4541,SECTION 1. RELIQUIDATION OF CERTAIN ENTRIES PR...,Requires the Customs Service to reliquidate ce...,To provide for reliquidation of entries premat...,11691,114
6,6,111_s1495,SECTION 1. SHORT TITLE.\n\n This Act may be...,Service Dogs for Veterans Act of 2009 - Direct...,A bill to require the Secretary of Veterans Af...,5328,379
7,7,111_s3885,SECTION 1. SHORT TITLE.\n\n This Act may be...,Race to the Top Act of 2010 - Directs the Secr...,A bill to provide incentives for States and lo...,16668,1525
8,8,113_hr1796,SECTION 1. SHORT TITLE.\n\n This Act may be...,Troop Talent Act of 2013 - Directs the Secreta...,Troop Talent Act of 2013,15352,2151
9,9,103_hr1987,SECTION 1. SHORT TITLE.\n\n This Act may be...,Taxpayer's Right to View Act of 1993 - Amends ...,Taxpayer's Right to View Act of 1993,5633,894


In [13]:
df_bills = df[['text', 'summary', 'title']]
df_bills

,text,summary,title
0,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...
1,SECTION 1. SHORT TITLE.\n\n This Act may be...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...
3,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...
4,SECTION 1. SHORT TITLE.\n\n This Act may be...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...
5,SECTION 1. RELIQUIDATION OF CERTAIN ENTRIES PR...,Requires the Customs Service to reliquidate ce...,To provide for reliquidation of entries premat...
6,SECTION 1. SHORT TITLE.\n\n This Act may be...,Service Dogs for Veterans Act of 2009 - Direct...,A bill to require the Secretary of Veterans Af...
7,SECTION 1. SHORT TITLE.\n\n This Act may be...,Race to the Top Act of 2010 - Directs the Secr...,A bill to provide incentives for States and lo...
8,SECTION 1. SHORT TITLE.\n\n This Act may be...,Troop Talent Act of 2013 - Directs the Secreta...,Troop Talent Act of 2013
9,SECTION 1. SHORT TITLE.\n\n This Act may be...,Taxpayer's Right to View Act of 1993 - Amends ...,Taxpayer's Right to View Act of 1993


In [14]:
# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df_bills['text'] = df_bills["text"].apply(lambda x : normalize_text(x))

/tmp/ipykernel_82976/3185142679.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bills['text'] = df_bills["text"].apply(lambda x : normalize_text(x))


In [15]:
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
df_bills['n_tokens'] = df_bills["text"].apply(lambda x: len(tokenizer.encode(x)))
df_bills = df_bills[df_bills.n_tokens<2000]
len(df_bills)

Token indices sequence length is longer than the specified maximum sequence length for this model (1480 > 1024). Running this sequence through the model will result in indexing errors
/tmp/ipykernel_82976/1732209292.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bills['n_tokens'] = df_bills["text"].apply(lambda x: len(tokenizer.encode(x)))


12

In [16]:
df_bills

,text,summary,title,n_tokens
0,SECTION 1. SHORT TITLE. This Act may be cited ...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...,1480
1,SECTION 1. SHORT TITLE. This Act may be cited ...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,1152
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...,930
4,SECTION 1. SHORT TITLE. This Act may be cited ...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...,1048
5,SECTION 1. RELIQUIDATION OF CERTAIN ENTRIES PR...,Requires the Customs Service to reliquidate ce...,To provide for reliquidation of entries premat...,1846
6,SECTION 1. SHORT TITLE. This Act may be cited ...,Service Dogs for Veterans Act of 2009 - Direct...,A bill to require the Secretary of Veterans Af...,872
9,SECTION 1. SHORT TITLE. This Act may be cited ...,Taxpayer's Right to View Act of 1993 - Amends ...,Taxpayer's Right to View Act of 1993,946
12,SECTION 1. FINDINGS. The Congress finds the fo...,Amends the Marine Mammal Protection Act of 197...,To amend the Marine Mammal Protection Act of 1...,1223
14,SECTION 1. SHORT TITLE. This Act may be cited ...,Education and Training for Health Act of 2017 ...,Education and Training for Health Act of 2017,1596
16,SECTION 1. SHORT TITLE. This Act may be cited ...,Andrew Prior Act or Andrew's Law - Amends the ...,Andrew's Law,608


In [17]:
understand_tokenization = tokenizer.tokenize(df_bills.text[0])
understand_tokenization

['S',
 'ECTION',
 'Ġ1',
 '.',
 'ĠSH',
 'ORT',
 'ĠTIT',
 'LE',
 '.',
 'ĠThis',
 'ĠAct',
 'Ġmay',
 'Ġbe',
 'Ġcited',
 'Ġas',
 'Ġthe',
 'Ġ``',
 'National',
 'ĠScience',
 'ĠEducation',
 'ĠTax',
 'ĠIn',
 'cent',
 'ive',
 'Ġfor',
 'ĠBusiness',
 'es',
 'ĠAct',
 'Ġof',
 'Ġ2007',
 "''.",
 'ĠSEC',
 '.',
 'Ġ2',
 '.',
 'ĠCR',
 'EDIT',
 'S',
 'ĠFOR',
 'ĠC',
 'ER',
 'TAIN',
 'ĠCONTR',
 'IB',
 'UT',
 'IONS',
 'ĠBEN',
 'EF',
 'IT',
 'ING',
 'ĠSC',
 'IENCE',
 ',',
 'ĠTECH',
 'N',
 'OLOGY',
 ',',
 'ĠENG',
 'INE',
 'ER',
 'ING',
 ',',
 'ĠAND',
 'ĠM',
 'ATH',
 'EM',
 'AT',
 'ICS',
 'ĠED',
 'UC',
 'ATION',
 'ĠAT',
 'ĠTHE',
 'ĠELE',
 'MENT',
 'ARY',
 'ĠAND',
 'ĠSEC',
 'OND',
 'ARY',
 'ĠSCHOOL',
 'ĠLEVEL',
 '.',
 'Ġ(',
 'a',
 ')',
 'ĠIn',
 'ĠGeneral',
 '.--',
 'Sub',
 'part',
 'ĠD',
 'Ġof',
 'Ġpart',
 'ĠIV',
 'Ġof',
 'Ġsub',
 'chapter',
 'ĠA',
 'Ġof',
 'Ġchapter',
 'Ġ1',
 'Ġof',
 'Ġthe',
 'ĠInternal',
 'ĠRevenue',
 'ĠCode',
 'Ġof',
 'Ġ1986',
 'Ġ(',
 'rel',
 'ating',
 'Ġto',
 'Ġbusiness',
 'Ġrelated',
 'Ġcred

In [18]:
len(understand_tokenization)

1480

In [20]:
df_bills['curie_search'] = df_bills["text"].apply(lambda x : from azure.core.credentials import AzureKeyCredential
(x, engine = 'text-search-curie-doc-001'))

/tmp/ipykernel_82976/1748282169.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bills['curie_search'] = df_bills["text"].apply(lambda x : get_embedding(x, engine = 'text-search-curie-doc-001'))


In [21]:
df_bills

,text,summary,title,n_tokens,curie_search
0,SECTION 1. SHORT TITLE. This Act may be cited ...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...,1480,"[-0.019770914688706398, 0.011169900186359882, ..."
1,SECTION 1. SHORT TITLE. This Act may be cited ...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,1152,"[-0.007850012741982937, 0.01001765951514244, 0..."
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...,930,"[0.00012103027984267101, 0.011845593340694904,..."
4,SECTION 1. SHORT TITLE. This Act may be cited ...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...,1048,"[-0.005481021944433451, 0.00856819562613964, -..."
5,SECTION 1. RELIQUIDATION OF CERTAIN ENTRIES PR...,Requires the Customs Service to reliquidate ce...,To provide for reliquidation of entries premat...,1846,"[-0.008310390636324883, -0.004660653416067362,..."
6,SECTION 1. SHORT TITLE. This Act may be cited ...,Service Dogs for Veterans Act of 2009 - Direct...,A bill to require the Secretary of Veterans Af...,872,"[-0.017687108367681503, 0.011164870113134384, ..."
9,SECTION 1. SHORT TITLE. This Act may be cited ...,Taxpayer's Right to View Act of 1993 - Amends ...,Taxpayer's Right to View Act of 1993,946,"[0.0021867561154067516, -0.004219848196953535,..."
12,SECTION 1. FINDINGS. The Congress finds the fo...,Amends the Marine Mammal Protection Act of 197...,To amend the Marine Mammal Protection Act of 1...,1223,"[-0.015813011676073074, 0.009919906966388226, ..."
14,SECTION 1. SHORT TITLE. This Act may be cited ...,Education and Training for Health Act of 2017 ...,Education and Training for Health Act of 2017,1596,"[-0.0150684155523777, 0.005073960404843092, 0...."
16,SECTION 1. SHORT TITLE. This Act may be cited ...,Andrew Prior Act or Andrew's Law - Amends the ...,Andrew's Law,608,"[-0.011593054980039597, 0.022752899676561356, ..."


In [22]:
# search through the reviews for a specific product
def search_docs(df, user_query, top_n=3, to_print=True):
    embedding = get_embedding(
        user_query,
        engine="text-search-curie-query-001"
    )
    df["similarities"] = df.curie_search.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res


res = search_docs(df_bills, "can i get information on cable company tax revenue", top_n=4)

/tmp/ipykernel_82976/1092181356.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["similarities"] = df.curie_search.apply(lambda x: cosine_similarity(x, embedding))


,text,summary,title,n_tokens,curie_search,similarities
9,SECTION 1. SHORT TITLE. This Act may be cited ...,Taxpayer's Right to View Act of 1993 - Amends ...,Taxpayer's Right to View Act of 1993,946,"[0.0021867561154067516, -0.004219848196953535,...",0.363270
0,SECTION 1. SHORT TITLE. This Act may be cited ...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...,1480,"[-0.019770914688706398, 0.011169900186359882, ...",0.314105
1,SECTION 1. SHORT TITLE. This Act may be cited ...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,1152,"[-0.007850012741982937, 0.01001765951514244, 0...",0.297908
18,SECTION 1. SHORT TITLE. This Act may be cited ...,This measure has not been amended since it was...,Veterans Entrepreneurship Act of 2015,1404,"[-0.020315825939178467, 0.0011716989101842046,...",0.295586


In [23]:
res["summary"][9]

"Taxpayer's Right to View Act of 1993 - Amends the Communications Act of 1934 to prohibit a cable operator from assessing separate charges for any video programming of a sporting, theatrical, or other entertainment event if that event is performed at a facility constructed, renovated, or maintained with tax revenues or by an organization that receives public financial support. Authorizes the Federal Communications Commission and local franchising authorities to make determinations concerning the applicability of such prohibition. Sets forth conditions under which a facility is considered to have been constructed, maintained, or renovated with tax revenues. Considers events performed by nonprofit or public organizations that receive tax subsidies to be subject to this Act if the event is sponsored by, or includes the participation of a team that is part of, a tax exempt organization."